# qubits

In [85]:
import cirq

def deutsch_jozsa_oracle(circuit, q0,q1, function_type='balanced'):
    # For a constant function that always returns 0, do nothing (i.e., identity)
    if function_type == 'constant_0':
        return 
    # For a constant function that always returns 1, flip the answer qubit
    elif function_type == 'constant_1':
        return circuit.append(cirq.X(q1))
    # For a balanced function, as an example, you might use a controlled-NOT
    # Here we show a simple controlled-NOT from one of the query qubits to the answer qubit
    elif function_type == 'balanced':
        return circuit.append(cirq.CNOT(q0, q1))
        
def deutsch_jozsa_algo(function):
    # Create a single qubit at position 0 on a 1D line
    qubit0 = cirq.LineQubit(0)
    qubit1 = cirq.LineQubit(1)

    # Create a circuit
    circuit = cirq.Circuit()

    circuit.append([cirq.X(qubit1)])
    circuit.append([cirq.H(qubit0), cirq.H(qubit1)])

    # Append the oracle operations to the circuit
    deutsch_jozsa_oracle(circuit, qubit0, qubit1,function)

    circuit.append([cirq.H(qubit0)])

    print(circuit)
    # Create a simulator
    simulator = cirq.Simulator()

    # Simulate the circuit
    result = simulator.simulate(circuit)
    # Print the state vector in LaTeX format
    print("\nState vector in LaTeX format:")
    print(cirq.dirac_notation(result.final_state_vector, decimals=3))
    
deutsch_jozsa_algo('balanced')
deutsch_jozsa_algo('constant_0')
deutsch_jozsa_algo('constant_1')

0: ───H───────@───H───
              │
1: ───X───H───X───────

State vector in LaTeX format:
0.707|10⟩ - 0.707|11⟩
0: ───H───H───

1: ───X───H───

State vector in LaTeX format:
0.707|00⟩ - 0.707|01⟩
0: ───H───H───────

1: ───X───H───X───

State vector in LaTeX format:
-0.707|00⟩ + 0.707|01⟩


# qudits

In [89]:
import cirq
from qudity import quditXGate,quditHGate, quditCNOTGate, state_vector, qudit_measure

# Set qudit dimension (e.g. 3 for qutrits)
d = 3

# Number of query qudits (r)
num_query = 1

# Create qudits: query register and one answer register qudit
query_qudits = [cirq.LineQid(i, dimension=d) for i in range(num_query)]
answer_qudit = cirq.LineQid(num_query, dimension=d)

# Build the circuit
circuit = cirq.Circuit()

# 1. Initialize the answer qudit to |d-1> (e.g. |2> for qutrits) by applying quditXGate (d-1) times
for _ in range(d - 1):
    circuit.append(quditXGate(d=d).on(answer_qudit))

# 2. Apply the generalized Hadamard (QFT) to all qudits (query + answer)
for q in query_qudits + [answer_qudit]:
    circuit.append(quditHGate(d=d).on(q))

# 3. Oracle: Apply a balanced oracle.
# Here we use a controlled operation from the first query qudit that adds 1 modulo d to the answer qudit.
circuit.append(quditCNOTGate(d=d).on(query_qudits[0], answer_qudit))

# 4. Apply the generalized Hadamard to the query qudits (not the answer qudit)
for q in query_qudits:
    circuit.append(quditHGate(d=d).on(q))

# 5. Measure the query qudits
for i, q in enumerate(query_qudits):
    circuit.append(cirq.measure(q, key=f'q{i}'))

print("Qudit Deutsch–Jozsa circuit:")
print(circuit)

simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=1)
print("Measurement results:")
print(result)


Qudit Deutsch–Jozsa circuit:
0 (d=3): ───H(d=3)─────H(d=3)─────M('q0')───

1 (d=3): ───X^1(d=3)───X^1(d=3)───H(d=3)────
Measurement results:
q0=0
